### 20184645 컴퓨터공학과 김영찬
### prog_webDB_alba

In [ ]:
# 크롤링에 필요한 각 라이브러리, 모둘 임포트 및 설치
#!pip install selenium
#!pip install bs4
#!pip install flask
import re # 정규표현식을 위한 임포트
import time # 웹 페이지가 뜨는 동안 시간지연을 하기 위해 임포트
import urllib
from selenium import webdriver
from bs4 import BeautifulSoup
from flask import Flask,render_template, request
import sqlite3

In [ ]:
#플라스크 객체 생성
app = Flask(__name__)


In [ ]:
@app.route('/main')
def alba_login():
    return render_template('login.html')

# 로그인 처리
@app.route('/login', methods=['POST'])
def login():
# 폼 입력값 가져오기
    idn = request.form['id']
    pwd = request.form['passwd']
    
    # 알바몬 웹 사이트 키기
    global browser 
    browser = webdriver.Chrome("./chromedriver.exe")
    browser.get("https://www.albamon.com/main")
    # 맨 처음 홈페이지에 들어가고 기업/개인 을 선택하는 팝업에서 개인을 클릭
    main=browser.find_element_by_id("btn_MType")
    main.click()
    # 로그인 페이지 이동
    browser.find_element_by_xpath("//*[@id='subContentbg']/div[1]/div[2]/div[1]/ul/li[1]/a").click()
    time.sleep(3)
    browser.switch_to.window(browser.window_handles[1])
    # idn pwd 입력
    browser.find_element_by_name('M_ID').send_keys(idn)
    browser.find_element_by_name('M_PWD').send_keys(pwd)
    browser.find_element_by_xpath("//*[@id='secMbr']/div/div[1]/form/div[2]/button").click()
    browser.switch_to.window(browser.window_handles[0])
    
    base_url = urllib.request.urlopen(browser.current_url).read()
    soup = BeautifulSoup(base_url,"html.parser")
    
    log = soup.find('span', class_='logArea')
    print(log)
    conn = sqlite3.connect('member.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    
    cursor.execute('select id from meminfo where id = ?',(idn,))
    row = cursor.fetchone()

    # 계정/비번 조사
    if log != None:
        if row == None:
            cursor.execute('''
            insert into meminfo (id, passwd) values(?, ?)''', 
                           (idn, pwd))
            conn.commit()
        return render_template('albaForm.html')
    # 안됐으면,   
    return("<h2>로그인 에러 !!</h2>")

In [ ]:
# 선수 입력 처리
@app.route('/insert', methods=['POST'])
def insert():
    browser.find_element_by_link_text("경기").click()
    # 폼 입력값 가져오기
    lo = request.form['loc']
    loc = request.form['location']
    d_loc = request.form['detail_location']
    outstr = "<h2>선수 입력</h2><br><hr>"
    return outstr

In [ ]:
if __name__ == '__main__':
    app.run()
   